In [31]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

In [32]:
class DataAnalysis:
    def __init__(self, num_trials=50):
        self.num_trials = num_trials
    
    def import_data(self, Spath, Upath):
        Sload = np.loadtxt(Spath, delimiter=',')
        Uload = np.loadtxt(Upath, delimiter=',')
        return Sload, Uload
    
    def generate_testvals(self, trial_version):
        RSS_mpc = []
        RSS_mhe = []
        SSU_mpc = []
        SSU_mhe = []
        num_failures = 0.0

        print(f"{"="*25} Generating Test Values {"="*25}")
        for trial in range(self.num_trials):
            try:
                Sref, Uref = self.import_data(f"trajectories/states/dircol_soln_{trial}.csv",
                                              f"trajectories/inputs/dircol_inpu_{trial}.csv")
                
                Smpc, Umpc = self.import_data(f"solves_MPC/states/{trial_version}/mpc_soln_{trial}.csv",
                                              f"solves_MPC/inputs/{trial_version}/mpc_inpu_{trial}.csv")
                
                Smhe, Umhe = self.import_data(f"solves_MHEMPC/states/{trial_version}/mhempc_soln_{trial}.csv",
                                              f"solves_MHEMPC/inputs/{trial_version}/mhempc_inpu_{trial}.csv")
                RSS_mpc.append(np.sum(np.square(Sref - Smpc)))
                RSS_mhe.append(np.sum(np.square(Sref - Smhe)))

                SSU_mpc.append(np.sum(np.square(Umpc)))
                SSU_mhe.append(np.sum(np.square(Umhe)))
                
            except Exception as e:
                print(f"Trial {trial} missing data from one or more controllers")
                num_failures += 1

        print(f"Failure Percentage: {num_failures/self.num_trials * 100}%")
        testval_dict = {"RSS_mpc": np.array(RSS_mpc),
                        "RSS_mhe": np.array(RSS_mhe),
                        "SSU_mpc": np.array(SSU_mpc),
                        "SSU_mhe": np.array(SSU_mhe)}
        return testval_dict
    
    def test_normality(self, a, tol=0.05):
        print(f"{"="*25} Testing Normality {"="*25}")
        s_stat, p_val = stats.shapiro(a)

        print(f"s = {s_stat:.5E} | p = {p_val:.5E}")

        if(p_val > tol):
            print(f"p_val = {p_val} > {tol}, the underlying distribution is likely normal")
            return True
        else: 
            print(f"p_val = {p_val} < {tol}, WARNING: the underlying distribution is likely not normal")
            return False

    def t_test(self, a, b):
        paired_diff = a - b
        if self.test_normality(paired_diff):
            print(f"{"="*25} Performing t-test {"="*25}")
            v_stat, v_val = stats.levene(a, b, center='median') #Brown-Forsythe test for equality of variance
            print(f"v = {v_stat:.5E} | p = {v_val:.5E}")
            t_stat, p_val = stats.ttest_rel(a, b)
            print(f"t = {t_stat:.5E} | p = {p_val:.5E}")
            return t_stat, p_val
        else:
            print(f"{"="*25} Performing Mann-Whitney U-test {"="*25}")
            u_stat, p_val = stats.mannwhitneyu(a, b)
            print(f"t = {u_stat:.5E} | p = {p_val:.5E}")
            return u_stat, p_val

In [33]:
num_trials = 50
trial_versions = ["base", "base_vel", "mismatch", "mismatch_vel"]
analyzer = DataAnalysis()

### RSS means t-test
Hypothesis: In an marine environment with external disturbances and model mismatch, a USV controlled by a MPC using a MHE to estimate the dynamics of the system will follow a trajectory with less error than a MPC using a dynamics model derived from calm water scale testing.

H0: There is no difference in the means of the RSS between the MHEMPC and the MPC (ie. MHEMPC and MPC perform similarly)

HA: There is a difference in the means of the RSS between the MHEMPC and the MPC (ie. MHEMPC and MPC perform differently)

In [34]:
for version in trial_versions:
    print(f"Data analysis for: {version}")
    testval_dict = analyzer.generate_testvals(version)
    RSS_mpc, RSS_mhe = testval_dict["RSS_mpc"], testval_dict["RSS_mhe"]
    t_stat, p_val = analyzer.t_test(RSS_mhe, RSS_mpc)

    print(f"{"="*25} RSS Means {"="*25}")
    RSS_mean_mpc = np.mean(RSS_mpc)
    RSS_mean_mhe = np.mean(RSS_mhe)
    print(f"MPC RSS Mean: {RSS_mean_mpc:.5E} | MHEMPC RSS Mean: {RSS_mean_mhe:.5E}")

    print("+"*100)

Data analysis for: base
========================= Generating Test Values =========================
Trial 13 missing data from one or more controllers
Trial 14 missing data from one or more controllers
Trial 40 missing data from one or more controllers
Trial 47 missing data from one or more controllers
Failure Percentage: 8.0%
========================= Testing Normality =========================
s = 4.83567E-01 | p = 1.67006E-11
p_val = 1.6700605562791492e-11 < 0.05, WARNING: the underlying distribution is likely not normal
========================= Performing Mann-Whitney U-test =========================
t = 2.02100E+03 | p = 5.64381E-14
========================= RSS Means =========================
MPC RSS Mean: 3.22531E-01 | MHEMPC RSS Mean: 1.31548E+01
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Data analysis for: base_vel
========================= Generating Test Values =========================
Trial 41 missing data from one 

### Average Actuator Effort
Hypothesis: In an marine environment with external disturbances and model mismatch, a USV controlled by a MPC using a MHE to estimate the dynamics of the system will expend less actuator effort to follow a trajectory than a MPC using a dynamics model derived from calm water scale testing.

In [35]:
for version in trial_versions:
    print(f"Data analysis for: {version}")
    testval_dict = analyzer.generate_testvals(version)
    SSU_mpc, SSU_mhe = testval_dict["SSU_mpc"], testval_dict["SSU_mhe"]
    t_stat, p_val = analyzer.t_test(SSU_mhe, SSU_mpc)

    print(f"{"="*25} Average Actuator Effort {"="*25}")
    SSU_mean_mpc = np.mean(SSU_mpc)
    SSU_mean_mhe = np.mean(SSU_mhe)
    print(f"MPC SSU Mean: {SSU_mean_mpc:.5E} | MHEMPC SSU Mean: {SSU_mean_mhe:.5E}")

    print("+"*100)

Data analysis for: base
========================= Generating Test Values =========================
Trial 13 missing data from one or more controllers
Trial 14 missing data from one or more controllers
Trial 40 missing data from one or more controllers
Trial 47 missing data from one or more controllers
Failure Percentage: 8.0%
========================= Testing Normality =========================
s = 7.10685E-01 | p = 3.31133E-08
p_val = 3.311326117365627e-08 < 0.05, WARNING: the underlying distribution is likely not normal
========================= Performing Mann-Whitney U-test =========================
t = 1.35100E+03 | p = 2.23650E-02
========================= Average Actuator Effort =========================
MPC SSU Mean: 3.17221E+03 | MHEMPC SSU Mean: 5.04134E+03
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Data analysis for: base_vel
========================= Generating Test Values =========================
Trial 41 missing d